In [1]:
#tworzę słownik trzymający [tag] : [wyrazy posiadające ten tag]
tag_dict = {}
with open('./../ruznezecy/supertags.txt', 'r', encoding='utf8') as supertags:
    for line in supertags:
        word, tag = line.split(" ")
        if tag[:-1] in tag_dict.keys():
            tag_dict[tag[:-1]].append(word)
        else:
            tag_dict[tag[:-1]] = [word]



In [2]:
#tworze słownik unigramowy [słowo] : [ilość wystąpień słowa]
unigram_dict = {}
with open('./../ruznezecy/1grams.txt', 'r', encoding='utf8') as unigrams:
    for line in unigrams:
        frequency, word = line.split()
        unigram_dict[word] = frequency


In [3]:
#tutaj łączę informację z dwóch słowników, aby mieć [tag] : [[wyraz1, częstość], [wyraz2, częstość], ...]
final_dict = {}
for tag in tag_dict:
    for word in tag_dict[tag]:
        if tag in final_dict.keys():
            if word in unigram_dict:
                final_dict[tag].append([word, unigram_dict[word]])
            else:
                final_dict[tag].append([word, "1"])
        else:
            if word in unigram_dict:
                final_dict[tag] = [[word, unigram_dict[word]]]
            else:
                final_dict[tag] = [[word, "1"]]


In [4]:
def end_of_word(word):
    return ('^' + word)[-3:]

def find_tags(sen):  # szukamy tagów dla każdego wyrazu w zdaniu
    result_tags = []
    word_set = sen.split()
    word_set = [word.lower() for word in word_set]
    for word in word_set:  
        found = 0
        for tag in tag_dict: # jeśli znajdziemy tag to nie ma problemu, dorzucamy do wyniku
            if word in tag_dict[tag]:
                result_tags.append(tag)
                found = 1
        if found == 0:
            for tag in tag_dict: #jeśli nie znajdziemy to korzystamy z podpowiedzi w poleceniu i dopasowujemy 3 ostatnie litery słowa
                if end_of_word(word) == end_of_word(tag_dict[tag][0]):
                    result_tags.append(tag)
                    break
    return result_tags

# print(find_tags("krek"))
print(find_tags("Witam państwa bardzo serdecznie"))

['T2693', 'T1385', 'T21', 'T21']


In [8]:
import random

def create_grammar_clone(sentence):
    result_sentence = []
    tags_of_words = find_tags(sentence) #bierzemy już wyliczone tagi
    for tag in tags_of_words:
        list_of_weights = []
        list_of_words = []
        for pair in final_dict[tag]: #dla każdego tagu dostajemy z final dict listę postaci [[słowo, częstość], [słowo, częstość], ...]
            list_of_weights.append(int(pair[1]))
            list_of_words.append(pair[0])
        result_sentence.append(random.choices(list_of_words, weights=list_of_weights, k=1)[0]) #według wyliczonego ppb wybieramy najlepszą opcję
    result_sentence[len(result_sentence) - 1] += "."
    result = ' '.join(result_sentence).capitalize()
    return ' '.join(result.split())

create_grammar_clone("Lubię obserwować ładne ptaki w piękne słoneczne dni w lesie")
# create_grammar_clone("Kupiłaś imbryk i postawiłaś go na kuchence")


'Lubię popierać odpowiedzialne gepardy w długodystansowe wielkie razy w tekście.'

In [ ]:
##################### ZADANIE 4

In [10]:
# Tutaj przygotowuję sobie słowniki składające się z klucz(słowo) : wartość[[następnik1:częstość występowania],...]
bigrams_weight_dict = {}
with open('./../ruznezecy/poleval_2grams.txt', 'r', encoding='utf8') as bigrams:
    for line in bigrams:
        current_line = line.split()
        frequency, first, second = current_line[0], current_line[1] , current_line[2]
        if int(frequency) > 5: #biorę tylko bigramy częstsze niż 5
            if first in bigrams_weight_dict.keys():
                bigrams_weight_dict[first].append([second, frequency])
            else:
                bigrams_weight_dict[first] = [[second, frequency]]
            

In [12]:
import itertools, random

def bigram_tag_maker(possible_bigrams):
    possible_bigrams_weights = []
    for pair_of_words in possible_bigrams:      #dla wszystkich permutacji podobnych gramatycznie słów tworzę słownik z bigramowymi wagami tych połączeń
        if pair_of_words[0] in bigrams_weight_dict:
            for frequ_pair in bigrams_weight_dict[pair_of_words[0]]:
                if pair_of_words[1] == frequ_pair[0]:
                    possible_bigrams_weights.append([[pair_of_words[0], pair_of_words[1]], frequ_pair[1]])
    return possible_bigrams_weights
    

def get_two_first_words(word1_tab, word2_tab):
    possible_bigrams = []
    for word1 in word1_tab:
        for word2 in word2_tab:
            possible_bigrams.append([word1, word2])
    possible_bigrams_weights = bigram_tag_maker(possible_bigrams)
    list_of_weights = []
    list_of_words = []
    for pair in possible_bigrams_weights:
        list_of_weights.append(int(pair[1]))   #tutaj wybieram najpopularniejsze z tych bigramów stworzonych z tagów
        list_of_words.append(pair[0])
    return random.choices(list_of_words, weights=list_of_weights, k=1)[0]

def match_another_word(current_word, new_word_set):
    possible_bigrams = []
    for word in new_word_set:
        possible_bigrams.append([current_word, word])
    possible_bigrams_weights = bigram_tag_maker(possible_bigrams)
    return possible_bigrams_weights
     

def create_grammar_bigram_clone(sentence):
    result_sentence = []
    tags_of_words = find_tags(sentence)
    word1_set = tag_dict[tags_of_words[0]] 
    word2_set = tag_dict[tags_of_words[1]] #tutaj biorę tagi pierwszych dwóch słów
    words_to_add = get_two_first_words(word1_set, word2_set)
    result_sentence.append(words_to_add[0])
    result_sentence.append(words_to_add[1])  #wrzucamy do wyniku wyliczone najpopularniejsze słowa
    current_word = words_to_add[1]     #teraz naszym current_word będzie słowo drugie i będziemy starali się dopasować do niego następne
    for i in range(1, len(tags_of_words) - 1):
        another_word_set = tag_dict[tags_of_words[i + 1]]
        possible_bigrams_weights =  match_another_word(current_word, another_word_set)
        if len(possible_bigrams_weights) > 0: #jeśli znajdę pasujące bigramy
            list_of_weights = [] 
            list_of_words = []
            for pair in possible_bigrams_weights:
                list_of_weights.append(int(pair[1]))   #tutaj wybieram najpopularniejsze z tych bigramów stworzonych z tagów
                list_of_words.append(pair[0])
            words_to_add = random.choices(list_of_words, weights=list_of_weights, k=1)[0]
            result_sentence.append(words_to_add[1])
            current_word = words_to_add[1]
        else:                                 #jeśli nie znajdę biramów to losuję z unigramów
            list_of_weights = []
            list_of_words = []
            for pair in final_dict[tags_of_words[i + 1]]:
                list_of_weights.append(int(pair[1]))
                list_of_words.append(pair[0])
            result_sentence.append("|")
            found_word = random.choices(list_of_words, weights=list_of_weights, k=1)[0]
            result_sentence.append(found_word)
            current_word = found_word


    result_sentence[len(result_sentence) - 1] += "."
    result = ' '.join(result_sentence).capitalize()
    return ' '.join(result.split())

In [13]:
create_grammar_bigram_clone("Trzeba mieć odwagę w domu")
# create_grammar_bigram_clone("Moja noga tutaj więcej nie postanie")

'Trzeba mieć siłę w wyniku.'